In [ ]:
#imports
!pip install skorch

from zipfile import ZipFile
import torch
torch.cuda.empty_cache()
torch.cuda.get_device_properties(0)
import os
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch import optim
import torch.nn as nn
from sklearn.metrics import accuracy_score
from skorch import NeuralNetClassifier , callbacks
from skorch.helper import predefined_split

In [ ]:
#uploads data
os.environ['KAGGLE_USERNAME'] = "XXXXX" #kaggle username
os.environ['KAGGLE_KEY'] = "XXXXX" #kaggle key
!kaggle datasets download -d ashishjangra27/gender-recognition-200k-images-celeba # api copied from kaggle

# Upload data and extract the contents

file_name = "/content/gender-recognition-200k-images-celeba.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()

 99% 1.31G/1.32G [00:31<00:00, 45.0MB/s]
100% 1.32G/1.32G [00:31<00:00, 45.0MB/s]


In [ ]:
#Use data augmentation to transforms.
def load_data(folder, dset):
    transform = {
        'train': transforms.Compose(
            [
                transforms.Resize([150, 150]),
                transforms.RandomCrop(130),
                transforms.ColorJitter(brightness = 0.5),
                transforms.RandomRotation(45),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor()]),
        'test': transforms.Compose(
            [transforms.ToPILImage(),
                transforms.Resize([130, 130]),
                transforms.ToTensor()]),
        'val':  transforms.Compose(
            [transforms.ToPILImage(),
                transforms.Resize([130, 130]),
                transforms.ToTensor()])
    }
    data = datasets.ImageFolder(
        root=folder, transform=transform['train' if dset else 'test'])
    return data


In [ ]:
train_data = '/content/Dataset/Train'
test_data = '/content/Dataset/Test'
val_data = '/content/Dataset/Validation'

train_set = load_data(train_data, 'train')
val_set = load_data(val_data, 'val')
test_set = load_data(test_data, 'test')

#Use data augmentation to increase the size of the training set.
  # from torchvision.utils import save_image
  # img_num = 0
  # for _ in range(2):
  #   for img in train_data:
  #     save_image(img, 'img' + str(img_num)+'.jpg')
  #     img_num += 1

In [ ]:
#A residual network, ResNet is an artificial neural network that helps to 
#build deeper neural network by utilizing skip connections or shortcuts to jump over some layers. 
model = resnet50(pretrained=True)
#Train and evaluate the model

#fc = remove the last FC layer from a ResNet model
model.fc = nn.Sequential(                  
                        nn.Linear(2048, 512),
                        nn.Dropout(0.5),
                        nn.Linear(512, 2),
                        nn.Dropout(0.5),
                        nn.Sigmoid()                        
                        )

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
#callback which is a learning rate scheduler that uses torch.optim.lr_scheduler.StepLR to scale learning rates by gamma=0.1 every 7 steps
from skorch.callbacks import LRScheduler
lrscheduler = LRScheduler(policy='StepLR', step_size=7, gamma=0.1)
from skorch.callbacks import Checkpoint
checkpoint = Checkpoint(f_params='best_model.pt', monitor='valid_acc_best')
from skorch.callbacks import ProgressBar
progress = ProgressBar(batches_per_epoch=1000)
#allows you to specify the performance measure to monitor, the trigger, and once triggered, it will stop the training process
from skorch.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='valid_loss', patience=5)
#opt using gradient descent
#The optimizer takes the parameters we want to update in SGD and can reduce the losses in much less effort.
optimizer = optim.SGD
#define loss function by CrossEntropyLoss
loss_funcion = nn.CrossEntropyLoss
#recognize patterns at the image
net = NeuralNetClassifier(
    model,
    #we use the optim package to define an Optimizer that will update the weights for us.
    optimizer=optimizer,
    optimizer__momentum=0.9,
    #computes the cross entropy loss between input and target.
    criterion=loss_funcion,
    max_epochs=10,
    #Transforms cardinal data to floats and categorical data to vectors of ints so that they can be embedded.
    train_split=predefined_split(val_set),
    iterator_train__shuffle=True,
    callbacks=[lrscheduler , progress , early_stop , checkpoint],
    #GPU
    device='cuda'

)

In [ ]:
#fit the model
net.fit(train_set, y=None)

  epoch    train_loss    valid_acc    valid_loss    cp      lr       dur
-------  ------------  -----------  ------------  ----  ------  --------
      1        0.5158       0.9615        0.3509     +  0.0100  799.5197


      2        0.5035       0.9549        0.3568        0.0100  793.9683


      3        0.5020       0.9723        0.3405     +  0.0100  793.3560


      4        0.5000       0.9748        0.3375     +  0.0100  793.6608


      5        0.4997       0.9755        0.3374     +  0.0100  793.3111


...

      7        0.4987       0.9783        0.3342        0.0100  793.8194


      8        0.4970       0.9807        0.3322     +  0.0010  793.6987


      9        0.4973       0.9813        0.3316     +  0.0010  793.8612


     10        0.4965       0.9809        0.3319        0.0010  795.1677


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=

In [ ]:
#acc val
predict = net.predict_proba(val_set)
testy = val_set.targets
acc_val = accuracy_score(testy, predict.argmax(axis=1))
print('Validation Accuracy: ',acc_val)

#acc tests
predict = net.predict_proba(test_set)
testy = test_set.targets
accur = accuracy_score(testy, predict.argmax(axis=1))
print('Test Accuracy: ', accur)

Validation Accuracy:  0.9811930259314984
Test Accuracy:  0.9862006899655017
